# Create All Revit Type Parameters Activities by Version

This notebook creates activities for multiple Revit versions (2021-2025) using the Autodesk Platform Services (APS) Design Automation API.

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [ ]:
import os
import json
import logging
import uuid
import time
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    ActivityJsonParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step2-create-forge-app/

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [ ]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned.
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

# Step 3: Define Revit Version Configurations

Configure all Revit versions (2021-2025) with their bundle paths and engines.

In [ ]:
# Base names (will have year appended)
app_bundle_base_name = "TypeParametersFromJsonDA"
activity_base_name = "TypeParametersActivity"
alias = "dev"  # Same alias for all versions

# Define configurations for each Revit version
revit_configs = [
    {"year": 2023, "engine": "Autodesk.Revit+2023", "zip_file": Path.cwd() / "2023 - 2024 - Net48" / "files" / "TypeParametersFromJsonDA.bundle.zip"},
    {"year": 2024, "engine": "Autodesk.Revit+2024", "zip_file": Path.cwd() / "2023 - 2024 - Net48" / "files" / "TypeParametersFromJsonDA.bundle.zip"},
    {"year": 2025, "engine": "Autodesk.Revit+2025", "zip_file": Path.cwd() / "2025 - 2026 - Net8" / "files" / "TypeParametersFromJsonDA.bundle.zip"},
    {"year": 2026, "engine": "Autodesk.Revit+2026", "zip_file": Path.cwd() / "2025 - 2026 - Net8" / "files" / "TypeParametersFromJsonDA.bundle.zip"},
]

print("Revit version configurations loaded:")
for config in revit_configs:
    print(f"  - Revit {config['year']}: {config['engine']}")

# Step 4: Create ZIP Files for All Bundles

Create ZIP files from bundle folders for each Revit version.

In [ ]:
# Verify ZIP file paths for each version's bundle
zip_files = {}

for config in revit_configs:
    year = config["year"]
    zip_file = config["zip_file"]
    
    if zip_file.exists():
        zip_files[year] = zip_file
        print(f"✓ Found ZIP for Revit {year}: {zip_file}")
    else:
        print(f"✗ ZIP file not found for Revit {year}: {zip_file}")

print(f"\n{len(zip_files)} ZIP files ready for deployment")

# Step 5: Deploy App Bundles for All Revit Versions

Deploy app bundles for each Revit version with version-specific aliases (21-25).

In [ ]:
# Deploy app bundles for all versions
deployed_bundles = {}

for config in revit_configs:
    year = config["year"]
    engine = config["engine"]
    
    if year not in zip_files:
        print(f"Skipping Revit {year} - no ZIP file available")
        continue
    
    # Add year to the bundle name, not the alias
    app_bundle_name = f"{app_bundle_base_name}{year}"
    
    try:
        bundle = AppBundle(
            appBundleId=app_bundle_name,
            engine=engine,
            alias=alias,
            zip_path=str(zip_files[year]),
            description=f"Type parameters from JSON for Revit {year}",
        )
        
        bundle.deploy(token)
        appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
        deployed_bundles[year] = {
            "name": app_bundle_name,
            "full_alias": appbundle_full_alias
        }
        
        print(f"✓ Deployed app bundle for Revit {year}: {appbundle_full_alias}")
        
    except Exception as e:
        error_msg = str(e)
        if "409" in error_msg or "Conflict" in error_msg:
            print(f"✓ App bundle already exists for Revit {year}: {app_bundle_name}")
            appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
            deployed_bundles[year] = {
                "name": app_bundle_name,
                "full_alias": appbundle_full_alias
            }
        else:
            print(f"✗ Failed to deploy app bundle for Revit {year}: {error_msg}")
    
    # Sleep to avoid overwhelming the APS server
    time.sleep(2)

print(f"\n{len(deployed_bundles)} app bundles ready (deployed or already existing)")

# Step 6: Deploy Activities for All Revit Versions

Create and deploy activities for each Revit version.

In [ ]:
# Deploy activities for all versions
deployed_activities = {}

for config in revit_configs:
    year = config["year"]
    engine = config["engine"]
    
    if year not in deployed_bundles:
        print(f"Skipping activity for Revit {year} - no deployed bundle")
        continue
    
    # Add year to the activity name, not the alias
    activity_name = f"{activity_base_name}{year}"
    bucket_key = uuid.uuid4().hex
    
    try:
        # Define activity parameters (same for all versions)
        input_revit = ActivityInputParameter(
            name="rvtFile",
            localName="input.rvt",
            verb="get",
            description="Input Revit model",
            required=True,
            is_engine_input=True,
            bucketKey=bucket_key,
            objectKey="input.rvt",
        )
        
        output_file = ActivityOutputParameter(
            name="result",
            localName="result.rvt",
            verb="put",
            description="Result Revit model",
            bucketKey=bucket_key,
            objectKey="result.rvt",
        )
        
        input_json = ActivityJsonParameter(
            name="configJson",
            localName="revit_type_params.json",
            verb="get",
            description="Type parameter JSON configuration",
        )
        
        # Create and deploy activity
        activity = Activity(
            id=activity_name,
            parameters=[input_revit, output_file, input_json],
            engine=engine,
            appbundle_full_name=deployed_bundles[year]["full_alias"],
            description=f"Create type parameters from JSON for Revit {year}",
            alias=alias,
        )
        
        activity.set_revit_command_line()
        activity.deploy(token=token)
        
        activity_full_alias = f"{nickname}.{activity_name}+{alias}"
        deployed_activities[year] = {
            "name": activity_name,
            "full_alias": activity_full_alias
        }
        
        print(f"✓ Deployed activity for Revit {year}: {activity_full_alias}")
        
    except Exception as e:
        error_msg = str(e)
        if "409" in error_msg or "Conflict" in error_msg:
            print(f"✓ Activity already exists for Revit {year}: {activity_name}")
            activity_full_alias = f"{nickname}.{activity_name}+{alias}"
            deployed_activities[year] = {
                "name": activity_name,
                "full_alias": activity_full_alias
            }
        else:
            print(f"✗ Failed to deploy activity for Revit {year}: {error_msg}")

print(f"\n{len(deployed_activities)} activities ready (deployed or already existing)")

# Step 7: Summary

Display all deployed app bundles and activities.

In [ ]:
# Display summary of deployed resources
print("=" * 80)
print("DEPLOYMENT SUMMARY")
print("=" * 80)

print(f"\nApp Bundles ({len(deployed_bundles)}):")
for year in sorted(deployed_bundles.keys()):
    print(f"  Revit {year}: {deployed_bundles[year]['full_alias']}")

print(f"\nActivities ({len(deployed_activities)}):")
for year in sorted(deployed_activities.keys()):
    print(f"  Revit {year}: {deployed_activities[year]['full_alias']}")

print("\n" + "=" * 80)
print("All activities created successfully!")
print("=" * 80)

# Step 8: Cleanup Resources (Optional)

Delete all deployed activities and app bundles to clean up resources.

In [ ]:
# Delete all activities for all versions
print("Deleting activities...")
for config in revit_configs:
    year = config["year"]
    activity_name = f"{activity_base_name}{year}"
    activity_full_alias = f"{nickname}.{activity_name}+{alias}"
    
    try:
        delete_activity(activityId=activity_name, token=token)
        print(f"✓ Deleted activity for Revit {year}: {activity_full_alias}")
    except Exception as e:
        print(f"✗ Failed to delete activity for Revit {year}: {e}")

# Delete all app bundles for all versions
print("\nDeleting app bundles...")
for config in revit_configs:
    year = config["year"]
    app_bundle_name = f"{app_bundle_base_name}{year}"
    bundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
    
    try:
        delete_appbundle(appbundleId=app_bundle_name, token=token)
        print(f"✓ Deleted app bundle for Revit {year}: {bundle_full_alias}")
    except Exception as e:
        print(f"✗ Failed to delete app bundle for Revit {year}: {e}")

print("\nCleanup completed!")